## Tabletop Game Recommendation

### Imports

In [3]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)

from IPython.display import Image, display, Markdown

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


### Functions

In [5]:
# Function to show the dimensions, column zero counts, column datatypes, column null counts, and the first and last 5 rows of the input dataframe
def check_df(df):
    # Display the dimensions of the DataFrame
    display(Markdown("#### DataFrame dimensions"))
    display(df.shape)

    # Add a new line
    display(Markdown("<br>"))

    # Display data types of all columns
    display(Markdown("#### Data Types, zeros and nulls"))
    display(pd.DataFrame({
        "Data type": df.dtypes,
        "Zero counts": (df == 0).sum(),
        "Zero count %": (((df == 0).sum()/df.count())* 100).round(2),
        "Null counts": df.isnull().sum(),
        "Null count %": ((df.isnull().sum())/(df.count()+df.isnull().sum())* 100).round(2)
    }))

# Function to return a dataframe with rows filtered by a specified word in a specified column:
def df_keyword_search(df, column, word):
    # Ensure the column is treated as strings and handle NaN values by replacing them with empty strings
    column_data = df[column].fillna('').astype(str)
    
    # Filter the DataFrame based on the presence of the word in the specified column, case insensitive
    keyword_df = df.loc[column_data.str.contains(rf'\b{word}\b', case=False, regex=True)]
    
    return keyword_df

# Function that converts a string to lowercase, and replaces whitespace with underscores, unless the whitespace is adjacent to ' // '
def format_text(words):
    # converts the string to lowercase
    result = str(words).lower()

    # replaces whitespace with underscores, unless the whitespace is adjacent to ' // '
    return(re.sub(r'(?<!//)\s(?!//)', '_',result))

# Function to replace '//' with ''
def remove_double_slashes(text):
    return text.replace(' // ', ' ')

# Function to count the different words
def count_words(df, column):
    # Ensure the column is treated as strings, handling NaN values
    column = df[column].fillna('').astype(str)
    
    # Concatenate all values in the column into a single string with '//' as a separator
    all_words_sep = column.str.cat(sep='//')
    
    # Split the single string into words based on the separator
    all_words = all_words_sep.split('//')
    
    # Convert the list of words into a pandas Series
    words_series = pd.Series(all_words)
    
    # Count occurrences of each word and return the result
    word_counts = words_series.value_counts()

    # Convert the Series to a DataFrame with words and their counts
    word_counts_df = word_counts.reset_index()
    word_counts_df.columns = ['Word', 'Count']

    # Calculate the percentage of total counts
    total_count = word_counts_df['Count'].sum()
    word_counts_df['Percentage'] = round((word_counts_df['Count'] / total_count) * 100,2)
    
    return word_counts_df

# Function to find all games in a dataframe which contain a term, and then take the average of the numerical values of a selected column for all returned games
def search_and_average(df, search_column, search_term, avg_column):
    # Filter the rows where the search term is found in the search column
    filtered_df = df[df[search_column].astype(str).str.contains(search_term, case=False, na=False)]
    
    # Exclude rows where the avg_column contains zero
    filtered_df = filtered_df[filtered_df[avg_column] != 0]
    
    # Calculate the average of the second column for the filtered rows
    avg_value = filtered_df[avg_column].mean()
    
    return avg_value

### Creating final dataframe

In [7]:
# Creating a final df from saved fe_df to use in generating the recommendations
final_df = pd.read_csv('Games_fe.csv')

### Creating a table of similarities

In [9]:
# Tokenize the bag of words based on whitespace separation
vectorizer = CountVectorizer(token_pattern=r'[^ ]+')

# Model learns the vocabulary from the text data ('bag of words'), and transforms the text data into a matrix of token counts
vectorized = vectorizer.fit_transform(final_df['bag of words'])

# Cosine similairty calculation, based on matrix of token counts
similarities = cosine_similarity(vectorized)

# score_df is used to contain the similarity matrix between all the games
score_df = pd.DataFrame(similarities, columns=final_df['name'], index=final_df['name'])

In [10]:
# Viewing the top 5 of score_df
score_df.head(5)

name                               Brass: Birmingham  \
name                                                   
Brass: Birmingham                           1.000000   
Pandemic Legacy: Season 1                   0.095443   
Gloomhaven                                  0.102598   
Ark Nova                                    0.233786   
Twilight Imperium: Fourth Edition           0.103905   

name                               Pandemic Legacy: Season 1  Gloomhaven  \
name                                                                       
Brass: Birmingham                                   0.095443    0.102598   
Pandemic Legacy: Season 1                           1.000000    0.248069   
Gloomhaven                                          0.248069    1.000000   
Ark Nova                                            0.251230    0.227866   
Twilight Imperium: Fourth Edition                   0.094211    0.202548   

name                               Ark Nova  \
name                                          
Brass: Birmingham                  0.233786   
Pandemic Legacy: Season 1          0.251230   
Gloomhaven                         0.227866   
Ark Nova                           1.000000   
Twilight Imperium: Fourth Edition  0.153846   

name                               Twilight Imperium: Fourth Edition  \
name                                                                   
Brass: Birmingham                                           0.103905   
Pandemic Legacy: Season 1                                   0.094211   
Gloomhaven                                                  0.202548   
Ark Nova                                                    0.153846   
Twilight Imperium: Fourth Edition                           1.000000   

name                               Dune: Imperium  Terraforming Mars  \
name                                                                   
Brass: Birmingham                        0.058272           0.254919   
Pandemic Legacy: Season 1                0.070447           0.196116   
Gloomhaven                               0.198787           0.180702   
Ark Nova                                 0.172559           0.388883   
Twilight Imperium: Fourth Edition        0.258839           0.183004   

name                               War of the Ring: Second Edition  \
name                                                                 
Brass: Birmingham                                         0.055641   
Pandemic Legacy: Season 1                                 0.067267   
Gloomhaven                                                0.162698   
Ark Nova                                                  0.054924   
Twilight Imperium: Fourth Edition                         0.192232   

name                               Star Wars: Rebellion  \
name                                                      
Brass: Birmingham                              0.106676   
Pandemic Legacy: Season 1                      0.161206   
Gloomhaven                                     0.233944   
Ark Nova                                       0.157949   
Twilight Imperium: Fourth Edition              0.421199   

name                               Gloomhaven: Jaws of the Lion  \
name                                                              
Brass: Birmingham                                      0.106676   
Pandemic Legacy: Season 1                              0.225689   
Gloomhaven                                             0.857794   
Ark Nova                                               0.210599   
Twilight Imperium: Fourth Edition                      0.184274   

name                               Spirit Island  Gaia Project  \
name                                                             
Brass: Birmingham                       0.162221      0.236842   
Pandemic Legacy: Season 1               0.228802      0.095443   
Gloomhaven                              0.342580      0.205196   
Ark Nova                     

### Generating a ranked similarity score

In [12]:
# Identifying the most similar games to the input game, based on the similarity matrix
input_game = 'Azul'

# Creating a dataframe called 'recommendations' which is ordered based on most similar to input game
recommendations = pd.DataFrame(score_df.nlargest(11,input_game)[input_game])

# Sort the DataFrame by similarity score in descending order
recommendations.sort_values(by=[input_game],ascending = False)

# Exclude the first row and select rows 1 to 11 & rename the 2nd column to 'Similarity score'
recommendations = recommendations.rename(columns={input_game: 'Similarity score'}).iloc[1:12]

In [13]:
recommendations

,Similarity score
name,
Azul: Summer Pavilion,0.857493
Azul: Stained Glass of Sintra,0.805161
Azul: Queen's Garden,0.805161
Azul: Master Chocolatier,0.727607
Project L,0.402200
Miyabi,0.400163
Splendor Duel,0.393445
Dragon Castle,0.377297
Harmonies,0.354246
